In [1]:
import spacy

In [2]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import jsonpickle
import json
import random
import tweepy
import spacy
import time
from datetime import datetime

In [3]:
class Tweets:
    
    
    def __init__(self,term="",corpus_size=100):
        self.tweets={}
        if term !="":
            self.searchTwitter(term,corpus_size)
                
    def searchTwitter(self,term,corpus_size):
        searchTime=datetime.now()
        while (self.countTweets() < corpus_size):
            new_tweets = api.search(term,lang="en",count=10)
            for nt_json in new_tweets:
                nt = nt_json._json
                if self.getTweet(nt['id_str']) is None and self.countTweets() < corpus_size:
                    self.addTweet(nt,searchTime,term)
            time.sleep(5)
                
    def addTweet(self,tweet,searchTime,term="",count=0):
        id = tweet['id_str']
        if id not in self.tweets.keys():
            self.tweets[id]={}
            self.tweets[id]['tweet']=tweet
            self.tweets[id]['count']=0
            self.tweets[id]['searchTime']=searchTime
            self.tweets[id]['searchTerm']=term
        self.tweets[id]['count'] = self.tweets[id]['count'] +1
        
    def getTweet(self,id):
        if id in self.tweets:
            return self.tweets[id]['tweet']
        else:
            return None
    
    def getTweetCount(self,id):
        return self.tweets[id]['count']
    
    def countTweets(self):
        return len(self.tweets)
    
    # return a sorted list of tupes of the form (id,count), with the occurrence counts sorted in decreasing order
    def mostFrequent(self):
        ps = []
        for t,entry in self.tweets.items():
            count = entry['count']
            ps.append((t,count))  
        ps.sort(key=lambda x: x[1],reverse=True)
        return ps
    
    # reeturns tweet IDs as a set
    def getIds(self):
        return set(self.tweets.keys())
    
    # save the tweets to a file
    def saveTweets(self,filename):
        json_data =jsonpickle.encode(self.tweets)
        with open(filename,'w') as f:
            json.dump(json_data,f)
    
    # read the tweets from a file 
    def readTweets(self,filename):
        with open(filename,'r') as f:
            json_data = json.load(f)
            incontents = jsonpickle.decode(json_data)   
            self.tweets=incontents
        
    def getSearchTerm(self,id):
        return self.tweets[id]['searchTerm']
    
    def getSearchTime(self,id):
        return self.tweets[id]['searchTime']
    
    def getText(self,id):
        tweet = self.getTweet(id)
        text=tweet['full_text']
        if 'retweeted_status'in tweet:
            original = tweet['retweeted_status']
            text=original['full_text']
        return text
                
    def addCode(self,id,code):
        tweet=self.getTweet(id)
        if 'codes' not in tweet:
            tweet['codes']=set()
        tweet['codes'].add(code)
        
   
    def addCodes(self,id,codes):
        for code in codes:
            self.addCode(id,code)
        
 
    def getCodes(self,id):
        tweet=self.getTweet(id)
        return tweet['codes']
    
    # NEW -ROUTINE TO GET PROFILE
    def getCodeProfile(self):
        summary={}
        for id in self.tweets.keys():
            tweet=self.getTweet(id)
            if 'codes' in tweet:
                for code in tweet['codes']:
                    if code not in summary:
                            summary[code] =0
                    summary[code]=summary[code]+1
        sortedsummary = sorted(summary.items(),key=operator.itemgetter(0),reverse=True)
        return sortedsummary

In [4]:
smoking=Tweets()
smoking.readTweets("tweets-smoking.json")
smoking.countTweets()

100

In [5]:
vaping=Tweets()
vaping.readTweets("tweets-vaping.json")
vaping.countTweets()

100

# ok. trying categorization from spaCy examples

https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py

In [6]:
nlp =spacy.blank("en")

In [7]:
textcat = nlp.create_pipe('textcat')

In [8]:
 nlp.add_pipe(textcat, last=True)

See - https://spacy.io/api/textcategorizer for info on text cat

In [9]:
textcat.add_label("POSITIVE")

1

In [10]:
import thinc.extra.datasets

In [11]:
from spacy.util import minibatch, compounding
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

## load data

In [12]:
train_data, _ = thinc.extra.datasets.imdb()

In [13]:
random.shuffle(train_data)

In [14]:
len(train_data)

25000

In [15]:
train_data[1]

("Definitely spoilers in this review! I **adore** American Gothic and have done since I first saw it late at night when it first aired on Ch4 in the UK when I was 14. The comparisons made to Stephen King are just about right. It's small town supernatural eeriness but with fantastic layered characters. Best of all, and the reason I love it so much, is it had the guts to never be black and white! Lucas Buck though lacking any conscience often works by, as he says, giving people enough rope to hang themselves with. His manipulation only works because of other people's weak morals. Caleb though generally a thoughtful, kind, insightful boy can at times show the latent dark side inherited from his father. None of the characters are wholly good or bad with even the angelic Merlyn showing a wrathful side through reckless vengeance in The Plague Sower. Not only that but having Gail, the closest thing to a mother figure for Caleb, not only sleep with but fall in love with Lucas despite all she k

In [16]:
 train_data = train_data[-2000:]

In [17]:
train_data[0]

("Riding Giants is an incredible documentary detailing the history and stories of three influential big-wave surfers, Gregg Noll, Jeff Clark, and Laird Hamilton. Stacy Peralta did an amazing job taking on the role of director and should be congratulated for doing such a brilliant job. The structure of the film is edited brilliantly and works perfectly with the narration, interviews, animation and surfing footage. The music soundtrack just adds to the overall satisfaction of watching this film, making Riding Giants brilliant viewing. Personal highlights include any of Greg Noll's comments, what with his straight-to-the-point frankness, Laird Hamilton's footage at Teahupoo, and the out-takes at the end of the movie. But really this entire film is one big, recommended highlight that comes highly recommended if you have the opportunity to see. It's a shame it isn't more well known, but it is a gem deserving of attention. 10/10",
 1)

split tuples into exts and labels

In [18]:
texts, labels = zip(*train_data)

In [19]:
texts[0]

"Riding Giants is an incredible documentary detailing the history and stories of three influential big-wave surfers, Gregg Noll, Jeff Clark, and Laird Hamilton. Stacy Peralta did an amazing job taking on the role of director and should be congratulated for doing such a brilliant job. The structure of the film is edited brilliantly and works perfectly with the narration, interviews, animation and surfing footage. The music soundtrack just adds to the overall satisfaction of watching this film, making Riding Giants brilliant viewing. Personal highlights include any of Greg Noll's comments, what with his straight-to-the-point frankness, Laird Hamilton's footage at Teahupoo, and the out-takes at the end of the movie. But really this entire film is one big, recommended highlight that comes highly recommended if you have the opportunity to see. It's a shame it isn't more well known, but it is a gem deserving of attention. 10/10"

In [20]:
labels[0]

1

In [21]:
cats = [{'POSITIVE': bool(y)} for y in labels]

## train on 80% of data

In [22]:
split = int(len(train_data) * 0.8)

In [23]:
split

1600

In [24]:
 (train_texts, train_cats), (dev_texts, dev_cats) = (texts[:split], cats[:split]), (texts[split:], cats[split:])

# ok. now have train texts and cats, along with dev..

rejoin them

In [25]:
train_data = list(zip(train_texts,
                          [{'cats': cats} for cats in train_cats]))

In [26]:
train_data[0]

("Riding Giants is an incredible documentary detailing the history and stories of three influential big-wave surfers, Gregg Noll, Jeff Clark, and Laird Hamilton. Stacy Peralta did an amazing job taking on the role of director and should be congratulated for doing such a brilliant job. The structure of the film is edited brilliantly and works perfectly with the narration, interviews, animation and surfing footage. The music soundtrack just adds to the overall satisfaction of watching this film, making Riding Giants brilliant viewing. Personal highlights include any of Greg Noll's comments, what with his straight-to-the-point frankness, Laird Hamilton's footage at Teahupoo, and the out-takes at the end of the movie. But really this entire film is one big, recommended highlight that comes highly recommended if you have the opportunity to see. It's a shame it isn't more well known, but it is a gem deserving of attention. 10/10",
 {'cats': {'POSITIVE': True}})

In [27]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']

In [28]:
nlp.disable_pipes(*other_pipes)

[]

In [29]:
optimizer = nlp.begin_training()

In [30]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [31]:
print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))
for i in range(2):
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_data, size=compounding(4., 32., 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(texts, annotations, sgd=optimizer, drop=0.2,losses=losses)
    with textcat.model.use_params(optimizer.averages):
        # evaluate on the dev data split off in load_data()
        scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
        print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'.format(losses['textcat'], scores['textcat_p'],scores['textcat_r'], scores['textcat_f']))
            

LOSS 	  P  	  R  	  F  
87.988	0.753	0.842	0.795
37.874	0.832	0.852	0.842


In [32]:
### questions - what does minibatch do? 
### what does nlp.update do? 
### what does the user_params do? 


### minibatch 

In [33]:
 batches = minibatch(train_data, size=compounding(4., 32., 1.001))

generates subsets.
NLP update updates the models. 
what about evaluate?

In [34]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [35]:
tokenizer= nlp.tokenizer

In [36]:
docs=(tokenizer(text) for text in dev_texts)

In [37]:
docs

<generator object <genexpr> at 0x10fad2f10>

In [38]:
enumerated=enumerate(textcat.pipe(docs))

In [39]:
elist=list(enumerated)

In [40]:
i,doc=elist[0]

In [41]:
gold=dev_cats[i]

In [42]:
gold

{'POSITIVE': True}

In [43]:
doc.cats

{'POSITIVE': 0.5923140048980713}

####  thus, gets a +1.
#### ahah. goes through all labels assocaited with the item in the training  data, and see how the tokenizer calls them. 


ok. so this is a bit opaque.  is sci-kit any clearer?

Trying the scikit spacy combination here ---

https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
import string
import re

In [45]:
train = ["I love space. Space is great.", "Planets are cool. I am glad they exist in space", 
        "lol @twitterdude that is gr8", "twitter &amp; reddit are fun.", 
        "Mars is a planet. It is red.", "@Microsoft: y u skip windows 9?", 
        "Rockets launch from Earth and go to other planets.", "twitter social media &gt; &lt;", 
        "@someguy @somegirl @twitter #hashtag", "Orbiting the sun is a little blue-green planet."]
labelsTrain = ["space", "space", "twitter", "twitter", "space", "twitter", "space", "twitter", "twitter", "space"]

test = ["i h8 riting comprehensibly #skoolsux", "planets and stars and rockets and stuff"]
labelsTest = ["twitter", "space"]

In [46]:
vectorizer = CountVectorizer()

In [47]:
vectorizer.transform(train)

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [ ]:
vectorizer.fit(train)

In [ ]:
print(vectorizer.vocabulary_)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
vectorizer.vocabulary_.get("windows")

In [ ]:
vector=vectorizer.transform(train)

In [ ]:
print(vector.shape)
print(type(vector))
print(vector.toarray())

In [ ]:
len(train)

https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(train)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

In [ ]:
vector = vectorizer.transform(train)
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

ok. now -- how can I train on this? 
and how can I pass in my spacy tokens?

In [ ]:
nlp =spacy.load("en")
def tokenizeText(sample):

    # get the tokens using spaCy
    tokens = nlp(sample)

    # stoplist the tokens
    tokens = [tok for tok in tokens if tok.is_stop ==False]
    # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas


    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

In [ ]:
tokenizeText(train[0])

In [ ]:
tokens=nlp(train[0])

In [ ]:
tokens

In [ ]:
tokens[0].is_stop

# plan

1. Go back to https://nicschrading.com/project/Intro-to-NLP-with-spaCy/

1.1 create NLP as per part 3 with hashtag tokenizer.

1.2. wrap that in a tokenizer that calls the spacy NLP parser and tokenizes with stops, etc.

1.3 use that tokenizer in a tf-df

1.4 follow that up with classifier. 

1.5 test


In [ ]:
nlp = spacy.load('en')
def hashtag_pipe(doc):
    merged_hashtag = True
    while merged_hashtag == True:
        merged_hashtag = False
        for token_index,token in enumerate(doc):
            if token.text == '#':
                try:
                    nbor = token.nbor()
                    start_index = token.idx
                    end_index = start_index + len(token.nbor().text) + 1
                    if doc.merge(start_index, end_index) is not None:
                        merged_hashtag = True
                        break
                except:
                    pass
    return doc
nlp.add_pipe(hashtag_pipe,first=True)

In [ ]:
def tokenizeText(sample):
    tokens = nlp(sample)
    
     # lemmatize
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas
    
    # stoplist the tokens
    tokens = [tok for tok in tokens if tok.is_stop == False]
    
    # remove large strings of whitespace
    while "" in tokens:
        tokens.remove("")
    while " " in tokens:
        tokens.remove(" ")
    while "\n" in tokens:
        tokens.remove("\n")
    while "\n\n" in tokens:
        tokens.remove("\n\n")

    return tokens

1.3 - set up tokenizer in a vectorizer.

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenizeText)

1.4 now I would do vectorizer. fit - need good input and labels..

In [ ]:
smoking

In [ ]:
vaping

## ok. to do this I think the following needs to happen:

1. take all tweets from smoking and vaping and put them into a list
2. make a similar list of smoking and vaping categories...

In [ ]:
ids=smoking.getIds()

In [ ]:
random.shuffle(ids)

In [ ]:
ids = list(ids)

In [ ]:
ids[0]

In [ ]:
random.shuffle(ids)

In [ ]:
ids[0]

In [ ]:
splitRatio=0.8
split = int(len(ids)*splitRatio)

In [ ]:
split

In [ ]:
trainSmoke=ids[:split]
testSmoke=ids[split:]

In [ ]:
len(trainSmoke)

In [ ]:
len(testSmoke)

In [ ]:
smoking.getSearchTerm(trainSmoke[0])

In [ ]:
def getTweetsAndCats(tweets,ids):
    texts=[]
    cats=[]
    for i in ids:
        text = tweets.getText(i)
        cat = tweets.getSearchTerm(i)
        texts.append(text)
        cats.append(cat)
    return texts,cats

def getTestTrain(tweets,splitFactor=0.8):
    ids =list(tweets.getIds())
    random.shuffle(ids)
    split = int(len(ids)*splitFactor)
    trainIds=ids[:split]
    testIds=ids[split:]
    trainTweets,trainCats = getTweetsAndCats(tweets,trainIds)
    testTweets,testCats = getTweetsAndCats(tweets,testIds)
    return trainTweets,trainCats,testTweets,testCats

In [ ]:
trainST,trainSC,testST,testSC = getTestTrain(smoking)

In [ ]:
trainVT,trainVC,testVT,testVC = getTestTrain(vaping)

In [ ]:
traint = trainST+trainVT
trainc = trainSC+trainVC
testt = testST+testVT
testc=testSC+testVC

In [ ]:
len(trainc)